In [1]:
import numpy as np
import pandas as pd
from glob import glob
import sys
import os
from datetime import datetime

import matplotlib.pyplot as plt

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import ccxt

sys.path.append("..") 
from tp_config import *
from tp_utils.data_provider import read_prices, read_data,load_data_from_exchange
from portfolio_tools import load_data_for_portfolio, calc_frontier, calc_weights, weights_to_df

In [12]:
import cvxpy
cvxpy.__version__

'1.4.1'

In [64]:
df_prices.index[-1]

1710950400000

## Helper functions

In [2]:
now = datetime.today().strftime('%Y-%m-%d')
def start_end_win(period, offset, horizon, lookback):
    start_win = offset + period * horizon
    end_win = start_win + lookback
    return (start_win, end_win)

def sell_portfolio(portfolio, sell_data, balance):
    if sell_data >= df_prices.shape[0]:
        sell_data = df_prices.shape[0] -1
        
    for key in portfolio.keys():
        asset_price = df_prices.iloc[sell_data][key]
        balance = balance + portfolio[key] * asset_price * 0.99925
    return balance

def buy_portfolio(buy_data, balance):
    overall_balance = balance
    for key in portfolio.keys():
  #      count = count +  portfolio[key]
        asset_price = df_prices.iloc[buy_data][key]

        sum_for_asset = portfolio[key] * overall_balance 
        balance = balance - min(sum_for_asset, balance) 
        quant = sum_for_asset / (asset_price * 1.00075)
        portfolio[key] = quant
        if balance < 0:
            print('Balance < 0')  
    return (portfolio, balance)

In [3]:
data_path = DATA_PATH_CRYPTO

pair = 'USDT'
#pair = 'BTC'

main_assets = [ 'BNB', 'THETA', 'RUNE', 'TFUEL', 'LINK', 'SOL', 'MATIC']
main_assets = [ 'BNB', 'AUDIO', 'RUNE', 'CTSI',  'TVK', 'POA', 'LUNA', 'TLM']
index_assets = pd.read_csv('index_assets.csv')['asset'].tolist()
#index_low_assets = pd.read_csv('index_low_assets.csv')['asset'].tolist()
assets = ['ADA', 'APT', 'AVAX', 'BNB', 'BONK', 'CYBER', 'ETH',  'ETHFI',  'NEON',  'WLD', 'PEPE', 'NEAR',  'WAVES',
          'FTT',  'FTM', 'MATIC', 'INJ', 'JUP', 'SEI', 'SUI', 'SOL', 'TRX', 'ZIL','THETA', 'RUNE', 'GALA', 
          'PYTH', 'PPT', 'FIRE', 'NGL']
assets.sort()
assets = index_assets
markets = [m + '-' + pair for m in assets]
if pair  == 'USDT':
    markets = ['BTC-USDT'] + markets

In [5]:
assets

['AAVE',
 'ADA',
 'ALGO',
 'APT',
 'AVAX',
 'AXS',
 'BNB',
 'BONK',
 'BOME',
 'CAKE',
 'CYBER',
 'DOGE',
 'DOT',
 'EOS',
 'ETC',
 'ETH',
 'ETHFI',
 'FIRE',
 'FTT',
 'FTM',
 'GALA',
 'INJ',
 'JUP',
 'LINK',
 'LTC',
 'LUNA',
 'MATIC',
 'NEON',
 'NEAR',
 'NGL',
 'ONDO',
 'PEPE',
 'PPT',
 'PYTH',
 'RUNE',
 'SEI',
 'SOL',
 'SUI',
 'THETA',
 'TOKEN',
 'TRX',
 'UNI',
 'WAVES',
 'WLD',
 'XRP',
 'ZIL']

In [9]:
### Data from exchange

In [6]:
exchange = ccxt.bybit()
start_timestamp = exchange.parse8601('2023-10-01 00:00:00')
timeframes = {'1h':60, '4h':240}

data_path = DATA_PATH_CRYPTO + '/' + pair
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, 10, verbose = False)

D:\Code\Projects\QuantMod\tp\Portfolio\..\tp_utils\data_provider.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])
D:\Code\Projects\QuantMod\tp\Portfolio\..\tp_utils\data_provider.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])


Done
Time taken = 0.010  hours


## Read from local data

In [4]:
df_all =  load_data_for_portfolio(markets, '4h')   
print(datetime.fromtimestamp(df_all.index[0]/1000))
print(datetime.fromtimestamp(df_all.index[-1]/1000))

BTC-USDT
AAVE-USDT
ADA-USDT
ALGO-USDT
APT-USDT
AVAX-USDT
AXS-USDT
BNB-USDT
BONK-USDT
BOME-USDT
CAKE-USDT
CYBER-USDT
DOGE-USDT
DOT-USDT
EOS-USDT
ETC-USDT
ETH-USDT
ETHFI-USDT
FIRE-USDT
FTT-USDT
FTM-USDT
GALA-USDT
INJ-USDT
JUP-USDT
LINK-USDT
LTC-USDT
LUNA-USDT
MATIC-USDT
NEON-USDT
NEAR-USDT
NGL-USDT
ONDO-USDT
PEPE-USDT
PPT-USDT
PYTH-USDT
RUNE-USDT
SEI-USDT
SOL-USDT
SUI-USDT
THETA-USDT
TOKEN-USDT
TRX-USDT
UNI-USDT
WAVES-USDT
WLD-USDT
XRP-USDT
ZIL-USDT
2023-10-01 04:00:00
2024-03-22 16:00:00


In [34]:
#global_scope = 84
since = 1694874400000
#df_prices = df_all[-global_scope:]
#df_prices = df_all
df_prices = df_all[df_all.index > since]

print(datetime.fromtimestamp(df_prices.index[0]/1000))
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

2023-10-01 04:00:00
2024-03-22 16:00:00


In [35]:
df_period = df_prices
ef = calc_frontier(df_period,  "semicovariance",  "ema_historical_return")
dfw = calc_weights(ef, 'max_sharpe', 0)   
dfw

,W,weights
BOME-USDT,0.55092,0.55092
ETHFI-USDT,0.44908,0.44908


In [10]:
window = 90
df_period = df_all[-window:]
ef = calc_frontier(df_period,  "semicovariance",  "ema_historical_return")
dfw = calc_weights(ef, 'max_sharpe', 0)   
dfw

,W,weights
JUP-USDT,0.99993,1.0


In [74]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

return_methods = [
    "mean_historical_return",
    "ema_historical_return",
    "capm_return", 
    ]

In [40]:
import riskfolio as rp


def riskfolio_weights(df_period, obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)

    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    rm = 'CVaR' # Risk measure used, this time will be variance
    obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
 #   port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
  #  port.nea = 2
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

In [41]:
lookback = 120 # 96, 95
horizon = 6 # 8, 10

res_trade = []
offset = 9 #np.random.randint(12)#,  8,9,10,11
n_periods = (df_prices.shape[0]  - lookback) // horizon
print('offset', offset)
balance = 100
portfolio = {}
for period in range(n_periods):

    start_win, end_win = start_end_win(period, offset, horizon, lookback)
  
    
#Calc frontier
    df_period = df_prices.iloc[start_win:end_win]
    df_period = df_period.dropna(axis = 1)
#    ef = calc_frontier(df_period, "semicovariance",  "ema_historical_return")
#    ef = calc_frontier(df_period, "ledoit_wolf",  "ema_historical_return", span = 150)
#    ef = calc_frontier(df_period, "sample_cov")
    dfw = calc_weights(ef, 'max_sharpe', 0)      
#    dfw = calc_weights(ef, 'efficient_risk', 0.7) 
    
    
    dfw = riskfolio_weights(df_period)

#Calc weights    
#    try:

        #dfw = calc_weights(ef, 'efficient_risk', 0.4) 
#        pass
#    except:

#continue
#    print(dfw)

#Buy portfolio    
    portfolio = dfw.to_dict()['weights']
    print(portfolio)
    
    if end_win < df_prices.shape[0] :
        portfolio, balance = buy_portfolio(end_win, balance)

    # Sell portfolio      
        sell_index = end_win + horizon
        sell_index = min(sell_index, df_prices.shape[0] - 1)
        balance = sell_portfolio(portfolio, sell_index, balance)
        cur_time = datetime.fromtimestamp(df_prices.index[sell_index]/1000)
        print(period, cur_time , balance)    

        res_trade.append(balance)

offset 9
{'BTC-USDT': 0.1868587684058344, 'ETH-USDT': 0.21698388022257212, 'INJ-USDT': 0.1306974588804328, 'NEON-USDT': 0.012276823451280963, 'TRX-USDT': 0.4531830670043503}
0 2023-10-23 16:00:00 104.31963208830788
{'BTC-USDT': 0.4718724022017298, 'FTT-USDT': 0.026683503523944496, 'LINK-USDT': 0.06916539669398346, 'TRX-USDT': 0.4202810151040225}
1 2023-10-24 16:00:00 109.30301840762769
{'BTC-USDT': 0.2364398875104425, 'ETH-USDT': 0.2924968746797142, 'NEON-USDT': 0.021716965379863944, 'TRX-USDT': 0.4493462722774967}
2 2023-10-25 16:00:00 112.2042559240077
{'BTC-USDT': 0.3542618371648544, 'THETA-USDT': 0.07141453779414338, 'TRX-USDT': 0.4049692212511677, 'XRP-USDT': 0.16935439884324183}
3 2023-10-26 16:00:00 109.64245628047908
{'TRX-USDT': 0.7310632157161396, 'XRP-USDT': 0.2689367822198193}
4 2023-10-27 16:00:00 111.25943551463317
{'BTC-USDT': 0.13701900791888136, 'TRX-USDT': 0.8309035418576556, 'XRP-USDT': 0.032077449883020845}
5 2023-10-28 16:00:00 111.82093765996284
{'BTC-USDT': 0.126

In [23]:
end_win

435

In [26]:
datetime.fromtimestamp(df_prices.index[end_win+4]/1000)

datetime.datetime(2021, 5, 23, 9, 0)

In [29]:
risk_methods = [
#    "ledoit_wolf_constant_variance",
    "ledoit_wolf",
#    "semicovariance"
]


return_methods = [
#    "mean_historical_return",
    "ema_historical_return",
    ]

## Mass cov

In [ ]:
%%time
res = []
offset = 0

for method in risk_methods: 
    print(method)
    for return_method in return_methods:
        for span in [300]:
            for lookback in range(82, 100):
                print(" ", lookback)
                for horizon in range(3,13):

                    n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
                    balance = 0.10
                    portfolio = {}
                    for period in range(n_periods):

                        start_win, end_win = start_end_win(period, offset, horizon, lookback)  

                    #Calc frontier
                        df_period = df_prices.iloc[start_win:end_win]
                        ef = calc_frontier(df_period, method,  return_method, span)
                        dfw = calc_weights(ef, 'max_sharpe', 0)      
                        #dfw = calc_weights(ef, 'efficient_risk', 0.35) 


                    #Buy portfolio    
                        portfolio = dfw.to_dict()['W']
                        #print(portfolio)
                        portfolio, balance = buy_portfolio(end_win, balance)

                    # Sell portfolio      
                        balance = sell_portfolio(portfolio, end_win + horizon, balance)
                        cur_time = datetime.fromtimestamp(df_prices.index[end_win+horizon]/1000)
                        #print(period, cur_time , balance)    

                    res.append([method, return_method, span, lookback, horizon, n_periods, balance])   

ledoit_wolf
  82


D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
D:\Soft\Anaconda\envs\t\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
D:\Soft\Anaconda\envs\t\Lib\

In [49]:
df = pd.DataFrame(res, columns = ['method', 'return_method', 'span', 'look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)
 
dfx.to_csv('index_01.csv', index = False)
#dfx = dfx[dfx['method'] != 'semicovariance']
#dfx.groupby('method').mean()
dfx[:20]

,method,return_method,span,look,horizon,n,balance
86,semicovariance,ema_historical_return,300,90,9,25,0.169693575
159,semicovariance,ema_historical_return,300,97,12,18,0.169340716
158,semicovariance,ema_historical_return,300,97,11,20,0.168435151
48,semicovariance,ema_historical_return,300,86,11,21,0.161159575
149,semicovariance,ema_historical_return,300,96,12,18,0.160226639
16,semicovariance,ema_historical_return,300,83,9,26,0.160004638
169,semicovariance,ema_historical_return,300,98,12,18,0.159507019
128,semicovariance,ema_historical_return,300,94,11,20,0.157484558
139,semicovariance,ema_historical_return,300,95,12,18,0.157375244
118,semicovariance,ema_historical_return,300,93,11,20,0.155565676


In [108]:
dfx.groupby(['look']).mean().sort_values('balance')

,span,horizon,n,balance
look,,,,
63,300.0,22.0,4.435897436,0.086803623
62,300.0,22.0,4.461538462,0.086866830
32,300.0,22.0,6.641025641,0.086943737
33,300.0,22.0,6.487179487,0.087032436
34,300.0,22.0,6.435897436,0.087556527
51,300.0,22.0,5.230769231,0.087705196
36,300.0,22.0,6.333333333,0.088396883
61,300.0,22.0,4.461538462,0.088443203
59,300.0,22.0,4.717948718,0.088556354


### Mass offset

In [284]:
res = []
offset = 0

for method in risk_methods: 
    print(method)
    for lookback in range(89, 92):
        print(" ", lookback)
        for horizon in [5,6,8, 9]:
            for offset in range(0,24):
                n_periods = (df_prices.shape[0]  - lookback - offset) // horizon - 1
                balance = 0.10
                portfolio = {}
                for period in range(n_periods):

                    start_win, end_win = start_end_win(period, offset, horizon, lookback)

                # Sell portfolio    

            #        print(balance)    
                    df_period = df_prices.iloc[start_win:end_win]
                    ef = calc_frontier(df_period, method)
                    try:
                        dfw = calc_weights(ef, 'max_sharpe', 0) 
                    except:
                        continue

                #Buy portfolio    
                    portfolio = dfw.to_dict()['W']
    #                print(portfolio)
                    portfolio, balance = buy_portfolio(end_win, balance)

                # Sell portfolio      
                    balance = sell_portfolio(portfolio, end_win + horizon, balance)
                    cur_time = datetime.fromtimestamp(df_prices.index[end_win+horizon]/1000)
    #                print(period, cur_time , balance)    


                    res.append([method, lookback, horizon, offset, n_periods, balance])        

semicovariance
  89
  90
  91
ledoit_wolf_constant_correlation
  89
  90
  91


In [292]:
df = pd.DataFrame(res, columns = ['method', 'look', 'horizon', 'offset','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx.groupby(['method','look', 'horizon' ]).mean().sort_values('balance')

offset             n  \
method                           look horizon                               
ledoit_wolf_constant_correlation 91   5        11.233065442  36.347876005   
                                 90   5        11.231735160  36.557077626   
                                      6        11.229281768  30.215469613   
                                      9        11.216560510  19.662420382   
                                 89   5        11.236095346  36.763904654   
                                 91   9        11.211538462  19.538461538   
                                      6        11.225000000  30.050000000   
                                      8        11.239171375  22.160075330   
                                 89   6        11.239010989  30.379120879   
                                 90   8        11.226591760  22.288389513   
                                 89   9        11.227848101  19.784810127   
semicovariance                   91   5        11.233065442  36.347876005   
ledoit_wolf_constant_correlation 89   8        11.219739292  22.415270019   
semicovariance                   91   6        11.225000000  30.050000000   
                                      9        11.211538462  19.538461538   
                                      8        11.239171375  22.160075330   
                                 90   6        11.229281768  30.215469613   
                                      9        11.216560510  19.662420382   
                                      5        11.231735160  36.557077626   
                                 89   5        11.236095346  36.763904654   
                                 90   8        11.226591760  22.288389513   
                                 89   9        11.227848101  19.784810127   
                                      6        11.239010989  30.379120879   
                                      8        11.219739292  22.415270019   

                                                   balance  
method                           look horizon               
ledoit_wolf_constant_correlation 91   5        0.127415394  
                                 90   5        0.127716651  
                                      6        0.127924961  
                                      9        0.127941528  
                                 89   5        0.127996540  
                                 91   9        0.128014798  
                                      6        0.128110338  
                                      8        0.128145928  
                                 89   6        0.128309196  
                                 90   8        0.128335053  
                                 89   9        0.128353634  
semicovariance                   91   5        0.128397841  
ledoit_wolf_constant_correlation 89   8        0.128462395  
semicovariance                   91   6        0.129114217  
                                      9        0.129166876  
                                      8        0.129203905  
                                 90   6        0.129546970  
                                      9        0.129672050  
                                      5        0.129681249  
                                 89   5        0.129838040  
                                 90   8        0.130105575  
                                 89   9        0.130506244  
                                      6        0.130511691  
                                      8        0.130532659

(291, 29)

In [274]:
df = pd.DataFrame(res, columns = ['method', 'look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[(dfx['look'] > 85) & (dfx['look'] < 92)]
dfx[:20]

,method,look,horizon,n,balance
1969,semicovariance,89,8,25,0.197796092
2139,semicovariance,90,9,22,0.193421789
2031,semicovariance,90,5,40,0.191268967
1767,semicovariance,88,6,34,0.190027756
2179,semicovariance,91,5,40,0.189497499
3578,semicovariance,100,9,21,0.188080246
3754,semicovariance,102,5,38,0.187885951
3755,semicovariance,102,5,38,0.187838461
1991,semicovariance,89,9,22,0.187554015
2387,semicovariance,92,7,28,0.187062521


In [228]:
dfx.groupby(['horizon']).mean().sort_values('balance')

,look,n,balance
horizon,,,
9,90.0,14.08,0.139155017
8,90.0,16.08,0.139321131
6,90.0,21.92,0.142085156
7,90.0,18.56,0.142215123
5,90.0,26.60,0.143235112


In [126]:
### Mass

In [ ]:
res = []
offset = 0

for lookback in range(72, 102):
    print(lookback)
    for horizon in range(5,11):

        n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
        balance = 0.10
        portfolio = {}
        all_balance = 0
        for period in range(n_periods):

            start_win, end_win = start_end_win(period, horizon, lookback)

        # Sell portfolio    

    #        print(balance)    
            df_period = df_prices.iloc[start_win:end_win]
            ef = calc_frontier(df_period, 'ledoit_wolf')
            try:
                dfw = calc_weights(ef, 'max_sharpe', 0) 
            except:
                continue

            balance = sell_portfolio(portfolio, end_win, balance)
            all_balance = balance
        #Buy portfolio    
            portfolio = dfw.to_dict()['W']
            portfolio, balance = buy_portfolio(end_win, balance)

        res.append([lookback, horizon, n_periods, all_balance])

In [ ]:
df = pd.DataFrame(res, columns = ['look', 'horizon','n', 'balance'])
dfx = df.sort_values('balance', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

In [230]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

In [ ]:
res = []
offset = 0
#0.35, 95, 10
for par in [0.35, 0.4, 0.45, 0.5]:
    for cov_method in risk_methods:
        for lookback in range(80, 100):
            print(lookback)
            for horizon in range(2,12):
                profit = 0
                n_periods = (df_prices.shape[0]  - lookback) // horizon - 1
                balance = 0.1
                portfolio = {}
                all_balance = 0
                for period in range(n_periods):

                    start_win, end_win = start_end_win(period, horizon, lookback)

                # Sell portfolio    

            #        print(balance)    
                    df_period = df_prices.iloc[start_win:end_win]
                    ef = calc_frontier(df_period, cov_method)
                    dfw = calc_weights(ef, 'efficient_risk', par) 
                    try:
                        pass
                    except:
                        continue

                    balance = sell_portfolio(portfolio, end_win, balance)
                    all_balance = balance
                    profit = profit + balance - 0.1
                #Buy portfolio    
                    balance = 0.10
                    portfolio = dfw.to_dict()['W']
                    portfolio, balance = buy_portfolio(end_win, balance)

                res.append([par, cov_method, lookback, horizon, n_periods, all_balance, profit])

In [232]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20] 

,par,cov,look,horizon,n,balance,profit
2625,0.40,ledoit_wolf_single_factor,82,7,20,0.107254666,0.076422891
2825,0.40,ledoit_wolf_constant_correlation,82,7,20,0.107145394,0.076284547
2904,0.40,ledoit_wolf_constant_correlation,90,6,22,0.107917221,0.075631286
2104,0.40,exp_cov,90,6,22,0.107943484,0.075624305
1625,0.40,sample_cov,82,7,20,0.107233674,0.075388251
2704,0.40,ledoit_wolf_single_factor,90,6,22,0.107904749,0.075289034
2813,0.40,ledoit_wolf_constant_correlation,81,5,28,0.103346873,0.075159280
1704,0.40,sample_cov,90,6,22,0.107906898,0.074867175
2877,0.40,ledoit_wolf_constant_correlation,87,9,14,0.132459390,0.074714529
4425,0.45,ledoit_wolf_constant_correlation,82,7,20,0.108042789,0.074680226


In [71]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

,par,cov,look,horizon,n,balance,profit
199,0.37,ledoit_wolf,95,10,91,0.102490153,0.316876595
149,0.37,ledoit_wolf,90,10,92,0.088793456,0.289672315
158,0.37,ledoit_wolf,91,9,102,0.092925225,0.289150017
222,0.37,ledoit_wolf,98,3,307,0.110125612,0.288139501
138,0.37,ledoit_wolf,89,9,102,0.089232278,0.285607226
153,0.37,ledoit_wolf,91,4,232,0.107691926,0.284661077
148,0.37,ledoit_wolf,90,9,102,0.087458152,0.284210233
46,0.37,ledoit_wolf,80,7,133,0.097544911,0.284045753
215,0.37,ledoit_wolf,97,6,153,0.101016831,0.281120509
128,0.37,ledoit_wolf,88,9,103,0.095319928,0.280706351


In [67]:
#dfw = calc_weights(ef, 'max_sharpe', 0) 
df = pd.DataFrame(res, columns = ['par', 'cov','look', 'horizon','n', 'balance', 'profit'])
dfx = df.sort_values('profit', ascending = False)

dfx.to_csv('index_01.csv', index = False)
dfx[:20]

,par,cov,look,horizon,n,balance,profit
199,0.35,ledoit_wolf,95,10,91,0.102404941,0.303427301
158,0.35,ledoit_wolf,91,9,102,0.092925225,0.278199292
222,0.35,ledoit_wolf,98,3,307,0.109691921,0.275417148
149,0.35,ledoit_wolf,90,10,92,0.088793456,0.274179610
138,0.35,ledoit_wolf,89,9,102,0.089232278,0.272283932
46,0.35,ledoit_wolf,80,7,133,0.097476395,0.271506323
148,0.35,ledoit_wolf,90,9,102,0.087458152,0.271206971
153,0.35,ledoit_wolf,91,4,232,0.107211998,0.270897136
128,0.35,ledoit_wolf,88,9,103,0.095319928,0.268818052
143,0.35,ledoit_wolf,90,4,232,0.102069821,0.267836961


In [47]:
portfolio = df_final.to_dict()['W']

buy_data = lookback
for key in portfolio.keys():
    asset_price = df_prices.iloc[buy_data][key]
    balance = balance - portfolio[key] * asset_price
    if balance < 0:
        print('Balance < 0')

for key in portfolio.keys():
    asset_price = df_prices.iloc[sell_data][key]
    balance = balance + portfolio[key] * asset_price
    if balance < 0:
        print('Balance < 0')

NameError: name 'buy_data' is not defined

## Basic

In [121]:
# calculate expected returns and sample covariance amtrix

avg_returns = expected_returns.mean_historical_return(df_prices)
cov_mat = risk_models.sample_cov(df_prices)
cov_mat;

In [11]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 3000)

Expected annual return: 225.7%
Annual volatility: 54.4%
Sharpe Ratio: 4.11


,W
LEND-USDT,660.0
REN-USDT,760.0
ADA-USDT,450.0
KAVA-USDT,1080.0


In [12]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 4000)

Expected annual return: -9.7%
Annual volatility: 26.3%
Sharpe Ratio: -0.45


,W
BTC-USDT,1850.0
BNB-USDT,2140.0
